**https://towardsdatascience.com/cleaning-preprocessing-text-data-by-building-nlp-pipeline-853148add68a**

In [ ]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 6.9 MB/s 


In [ ]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 622 kB 7.1 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=885dd1126abfd63a651b966b1afec9c5543efb0faaff1aab2b570b1bd4a7e0bd
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [ ]:
#https://towardsdatascience.com/cleaning-preprocessing-text-data-by-building-nlp-pipeline-853148add68a
# Importing Libraries 
# Importing Libraries
import unidecode
import pandas as pd
import re
import time
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
import timeit

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount("/Cleaning")

Mounted at /Cleaning


In [ ]:
# Read Dataset
Df = pd.read_csv('/Cleaning/MyDrive/NLP/Abdullah/Restuarants Reviews Final.csv', encoding = 'latin-1')
print('Number of Data points : ', Df.shape[0])
print('Number of features :', Df.shape[1])
print('features :', Df.columns.values)
# Show Dataset
Df.head()

Number of Data points :  1000
Number of features : 1
features : ['Review']


,Review
0,I had an amazing experience here in my short s...
1,I was 100 satisfied with the stay at intercont...
2,All of the facilities are modern & updated her...
3,I have been impressed by the excellent hospita...
4,"Featuring 226 luxury rooms and suites, a selec..."


In [ ]:
# This command tells information about the attributes of Dataset.
Df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [ ]:
# Shows statistics for every numerical column in our dataset.
Df.describe()

,Review
count,1000
unique,995
top,Good
freq,3


**Step-1 : Remove newlines & Tabs:**

You may encounter lots of new lines for no reason in your textual dataset and tabs as well. So when you scrape data, those newlines and tabs that are required on the website for structured content are not required in your dataset and also get converted into useless characters like \n, \t. So, I have written a function that will delete all such nonsense.

In [ ]:

def remove_newlines_tabs(text):
    """
    This function will remove all the occurrences of newlines, tabs, and combinations like: \\n, \\.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of newlines, tabs, \\n, \\ characters.
        
    Example:
    Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
    Output : This is her first day at this place. Please, Be nice to her. 
    
    """
    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text
# len of data :- 1618647 lac words

In [ ]:

text="This is her \\ first day at this place.\n Please,\t Be nice to her.\\n"
remove_newlines_tabs(text)


'This is her   first day at this place.  Please,  Be nice to her. '

**Step-2 : Strip HTML Tags**

When you scrape data, you may end up seeing HTML tags in the text of your dataset if you haven’t deal with it already while scraping. So, there is a need to deal with those tags later on. In this function, I am removing everything that matches HTML tags in my text.

In [ ]:

def strip_html_tags(text):
    """ 
    This function will remove all the occurrences of html tags from the text.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of html tags.
        
    Example:
    Input : This is a nice place to live. <IMG>
    Output : This is a nice place to live.  
    """
    # Initiating BeautifulSoup object soup.
    soup = BeautifulSoup(text, "html.parser")
    # Get all the text other than html tags.
    stripped_text = soup.get_text(separator=" ")
    return stripped_text
# len of string:- 1616053 lac words

In [ ]:

input="This is a nice place to live. <IMG>"
strip_html_tags(input)


'This is a nice place to live. '

**Step-3: Remove Links**

This step will remove everything which is similar to hyperlinks of any type. I have added this function here as I have dealt with it on my dataset.

In [ ]:
def remove_links(text):
    """
    This function will remove all the occurrences of links.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of all types of links.
        
    Example:
    Input : To know more about this website: kajalyadav.com  visit: https://kajalyadav.com//Blogs
    Output : To know more about this website: visit:     
    
    """
    
    # Removing all the occurrences of links that starts with https
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com
    

In [ ]:

input="To know more about this website: kajalyadav.com  visit: https://kajalyadav.com//Blogs"
remove_links(input)


'To know more about this website:   visit: '

**Step-4: Remove Whitespaces**

A single line function can be performed to remove extra whitespaces as mentioned below. This step is crucial before performing further NLP tasks.

In [ ]:
def remove_whitespace(text):
    """ This function will remove 
        extra whitespaces from the text
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after extra whitespaces removed .
        
    Example:
    Input : How   are   you   doing   ?
    Output : How are you doing ?     
        
    """
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text 
    

In [ ]:

input="How   are   you   doing   ?"
remove_whitespace(input)


'How are you doing  ? '

**What are the main NLP text preprocessing steps?**

The below list of text preprocessing steps is really important and I have written all these steps in a sequence how they should be.

**Step-1: Remove Accented Characters**

This is a crucial step to convert all characters like accented characters into machine-understandable language. So that further steps can be implemented easily. Accented characters are characters like â, î, or ô which have diacritics above the characters.

In [ ]:

# Code for accented characters removal
def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text
    

In [ ]:

input="Málaga, àéêöhello"
accented_characters_removal(input)


'Malaga, aeeohello'

**Step-2: Case Conversion**

This is the next step in the series which is really important as case, CASE is two different words for the machine. So, you should be converting the case of the text in either lowercase or uppercase to proceed further.

In [ ]:

# Code for text lowercasing
def lower_casing_text(text):
    
    """
    The function will convert text into lower case.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text in lowercase
         
    Example:
    Input : The World is Full of Surprises!
    Output : the world is full of surprises!
    
    """
    # Convert text to lower case
    # lower() - It converts all upperase letter of given string to lowercase.
    text = text.lower()
    return text
    

In [ ]:

input="The World is Full of Surprises!"
lower_casing_text(input)


'the world is full of surprises!'

**Step-3: Reducing repeated characters and punctuations**

This step is important as there might be scenarios where characters are repeating more than necessary which can’t be detected by a spell checker later on. Therefore, there is a need to deal with such a scenario beforehand before applying the spell checker function. There is another scenario that I encounter while working, there can be repeating punctuations as well. So there is a need to encounter them as well.

When we are very exciting, we do overwrite things that overwhelm readers.

Example:- Cheeeeeerrrrrrss !!!!!!

**The explanation for using some symbols in the above regex expression**

\1 → is equivalent to re.search(…). group(1). It refers to the first capturing group. \1 matches the exact same text that was matched by the first capturing group.

{1,} → It means we are matching for repetition that occurs more than one time.

DOTALL -> It matches the newline character as well unlike the dot operator which matches everything in the given text except the newline character.

sub() → This function is used to replace occurrences of a particular sub-string with another sub-string. This function takes as input the following: The substring to replace. The sub-string to replace with.

r’\1\1' → It limits all the repetition to two characters.

r’\1' → Limits all the repetition to only one character.

{2,} → It means to match for repetition that occurs more than two times

In [ ]:
# Code for removing repeated characters and punctuations

def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

**The explanation for using some symbols in the above regex expression**

\1 → is equivalent to re.search(…). group(1). It refers to the first capturing group. \1 matches the exact same text that was matched by the first capturing group.

{1,} → It means we are matching for repetition that occurs more than one time.

DOTALL -> It matches the newline character as well unlike the dot operator which matches everything in the given text except the newline character.

sub() → This function is used to replace occurrences of a particular sub-string with another sub-string. This function takes as input the following: The substring to replace. The sub-string to replace with.

r’\1\1' → It limits all the repetition to two characters.

r’\1' → Limits all the repetition to only one character.

{2,} → It means to match for repetition that occurs more than two times

In [ ]:

Input = "Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)"
reducing_incorrect_character_repeatation(Input)


'Reallyy, Greeaatt !?.;:)'

**Step-4: Expand Contractions**

In order to remove Stop words in the next step, It is crucial that you deal with contractions first. Contractions are nothing but shorthand forms for words like, Do not, would not, It is. Contractions are anything similar to these examples don’t, wouldn’t, It’s.

In [ ]:

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}


In [ ]:
# The code for expanding contraction words
def expand_contractions(text, contraction_mapping =  CONTRACTION_MAP):
    """expand shortened words to the actual form.
       e.g. don't to do not
    
       arguments:
            input_text: "text" of type "String".
         
       return:
            value: Text with expanded form of shorthened words.
        
       Example: 
       Input : ain't, aren't, can't, cause, can't've
       Output :  is not, are not, cannot, because, cannot have 
    
     """
    # Tokenizing text into tokens.
    list_Of_tokens = text.split(' ')
    print("text : ",text)

    # Checking for whether the given token matches with the Key & replacing word with key's value.
    
    # Check whether Word is in lidt_Of_tokens or not.
    for Word in list_Of_tokens: 
        # Check whether found word is in dictionary "Contraction Map" or not as a key. 
         if Word in CONTRACTION_MAP: 
                # If Word is present in both dictionary & list_Of_tokens, replace that word with the key value.
                list_Of_tokens = [item.replace(Word, CONTRACTION_MAP[Word]) for item in list_Of_tokens]
                
    # Converting list of tokens to String.
    String_Of_tokens = ' '.join(str(e) for e in list_Of_tokens) 
    return String_Of_tokens     

In [ ]:

Input = "ain't aren't can't cause can't've"
print(expand_contractions(Input,CONTRACTION_MAP ))


text :  ain't aren't can't cause can't've
is not are not cannot cause cannot've


In [ ]:
print(Input.split(' '))

["ain't", "aren't", "can't", 'cause', "can't've"]


**Step-5: Remove Special Characters**

In this step, We will go to learn how to remove special characters, why to remove them, and what special characters you should be keeping.

So, I have written a function that will remove a set of specified special characters and will gonna keep some important punctuations like (,.?!) excluding brackets. Special characters should be removed as when we gonna tokenized the text, later on, the punctuations don’t occur with more weightage.

**To Remove numbers from the text :**

All you need is to modify the given regex to

**Formatted_Text = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', text)**

Just exclude the 0–9 range so as to remove all representation of numbers from the text. I didn’t perform this particular step on my dataset as the numbers are really important in my case to consider.

Punctuations that I am considering are Important as per my Dataset as I have to perform text summarization later on.

,.?! → These are some frequent punctuations that occur a lot and needed to be preserved to understand the context of the text.

: → This one is also frequent as per the Dataset. It is important to keep because it is giving sense whenever there is an occurrence of time like 9:05 p.m.

% → This one is also frequently used in many articles and telling more precisely about the data, facts & figures.

$ → This one is used in many articles where prices are considered. So, omitting this symbol will not give much sense about those prices that are left as just some numbers only.

In [ ]:

# The code for removing special characters
def removing_special_characters(text):
    """Removing all the special characters except the one that is passed within 
       the regex to match, as they have imp meaning in the text provided.
   
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text with removed special characters that don't require.
        
    Example: 
    Input  : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
    Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
   """
    # The formatted text after removing not necessary punctuations.
    Formatted_Text = re.sub(r"[^a-zA-Z0-9$-,%.?!]+", ' ', text)
    #print("Formatted_Text : ",Formatted_Text)
    # In the above regex expression,I am providing necessary set of punctuations that are frequent in this particular dataset.
    return Formatted_Text


In [ ]:

Input = "Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?)"
removing_special_characters(Input)


'Hello, K a j a l. Thi*s is $100.05 the payment that you will recieve! (Is this okay?)'

**Step-7: Correcting Mis-spelled words**

You should be very careful while attempting this one step. As this function may change the true meaning of the word. So you have to be very careful and try to see how things unfold on applying this function. If you are working on some industry-specific dataset, then you may need to consider relating the dictionary which tells this function explicitly to keep those specific words as it is.

In [ ]:

# The code for spelling corrections
def spelling_correction(text):
    ''' 
    This function will correct spellings.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after corrected spellings.
        
    Example: 
    Input : This is Oberois f3rom Dlhi whoop came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.
      
    
    '''
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text


In [ ]:

Input = "This is Oberois from Dlhi who came heree to studdy."
spelling_correction(Input)


'This is Oberoi from Delhi who came here to study.'